In [25]:
import os
os.chdir("..")
from loaders import load_resisc45
from models import CropAndResize, SRResNet, SRGAN
import numpy as np
import keras
from keras.layers import RandomCrop, Resizing, DepthwiseConv2D

In [26]:
dataset, labels = load_resisc45(r"C:\Users\nedst\Desktop\synoptic-project-NedStickler\.venv\Lib\site-packages\tensorflow_datasets")
batch = dataset[:32]

In [ ]:
# https://stackoverflow.com/questions/55643675/how-do-i-implement-gaussian-blurring-layer-in-keras#:~:text=%23%20somewhere%20in%20your%20model%20you,of%20this%20layer%20%23%20the%20rest
# https://stackoverflow.com/questions/17190649/how-to-obtain-a-gaussian-filter-in-python/17201686#17201686

In [62]:
from keras.layers import DepthwiseConv2D

class GaussianBlur(DepthwiseConv2D):
    def __init__(self, kernel_size):
        super().__init__(kernel_size)

    def _matlab_style_gauss2D(self, shape=(3,3), sigma=0.5):
        """
        2D gaussian mask - should give the same result as MATLAB's
        fspecial('gaussian',[shape],[sigma])
        """
        m,n = [(ss-1.)/2. for ss in shape]
        y,x = np.ogrid[-m:m+1,-n:n+1]
        h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
        h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
        sumh = h.sum()
        if sumh != 0:
            h /= sumh
        return h
    
    def build(self, input_size):
        super().build(input_size)
        gaussian_kernel = self._matlab_style_gauss2D()
        kernel_weights = np.expand_dims(gaussian_kernel, axis=-1)
        kernel_weights = np.repeat(kernel_weights, 3, axis=-1) # apply the same filter on all the input channels
        kernel_weights = np.expand_dims(kernel_weights, axis=-1)
        self.depthwise_conv.set_weights([kernel_weights])
        self.depthwise_conv.trainable = False

In [64]:
GaussianBlur(3)(batch)

AttributeError: 'GaussianBlur' object has no attribute 'depthwise_conv'

In [69]:
class CropAndResize(keras.Model):
    def __init__(self, downsample_factor):
        super().__init__()
        self.downsample_factor = downsample_factor
        self.random_crop = RandomCrop(96, 96)
        self.gaussian_blur = DepthwiseConv2D(3, padding="same", use_bias=False)
        self.gaussian_blur.build()
        self.resize = Resizing(96 // downsample_factor, 96 // downsample_factor, interpolation="bicubic")

    def _matlab_style_gauss2D(self, shape=(3,3), sigma=0.5):
        """
        2D gaussian mask - should give the same result as MATLAB's
        fspecial('gaussian',[shape],[sigma])
        """
        m,n = [(ss-1.)/2. for ss in shape]
        y,x = np.ogrid[-m:m+1,-n:n+1]
        h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
        h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
        sumh = h.sum()
        if sumh != 0:
            h /= sumh
        return h
    
    def get_config(self):
        return {"downsample_factor": self.downsample_factor}
    
    def call(self, inputs):
        # hr_patch = self.random_crop(inputs)
        # blurred_hr_patch = self.gaussian_blur(hr_patch)
        # lr_patch = self.resize(blurred_hr_patch)
        # return lr_patch, hr_patch
        gaussian_kernel = self._matlab_style_gauss2D()
        kernel_weights = np.expand_dims(gaussian_kernel, axis=-1)
        kernel_weights = np.repeat(kernel_weights, 3, axis=-1) # apply the same filter on all the input channels
        kernel_weights = np.expand_dims(kernel_weights, axis=-1)
        self.gaussian_blur.set_weights([kernel_weights])
        self.gaussian_blur.trainable = False
        
        return self.gaussian_blur(inputs)

In [71]:
cr = CropAndResize(4)
cr.gaussian_blur.get_weights()

[]